## Download File from S3

In [17]:
import boto3

# Initialize Boto3 S3 client
s3 = boto3.client('s3')

# Specify the S3 bucket name and the path to your model file
bucket_name = 'lpr-characters-46000'
model_key = 'LPR_Character_Classify.tar.gz'
local_model_path = 'LPR_Character_Classify.tar.gz'

# Download the model from S3
s3.download_file(bucket_name, model_key, local_model_path)

## Upload .tar.gz to S3

In [21]:
import sagemaker
import torch
import tarfile

with tarfile.open("model.tar.gz", "w:gz") as f:
    f.add("classify_chars.pt")

In [51]:
import boto3
import sagemaker
import time
from sagemaker.utils import name_from_base

role = sagemaker.get_execution_role()
sess = sagemaker.Session()

region = sess.boto_region_name
bucket = sess.default_bucket()

compilation_job_name = name_from_base("LPRThaiCharsClassify")
prefix = compilation_job_name + "/model"

model_path = sess.upload_data(path="model.tar.gz", key_prefix=prefix)



sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [52]:
data_shape = '{"input0":[1,3,48,36]}'
target_device = "ml_c5"
framework = "PYTORCH"
framework_version = "1.13"
compiled_model_path = "s3://{}/{}".format(bucket, compilation_job_name)
model_path

's3://sagemaker-us-east-1-058264315246/LPRThaiCharsClassify-2024-02-12-08-27-53-874/model/model.tar.gz'

In [53]:
from sagemaker.pytorch.model import PyTorchModel
from sagemaker.predictor import Predictor

sagemaker_model = PyTorchModel(
    model_data=model_path,
    predictor_cls=Predictor,
    framework_version=framework_version,
    role=role,
    sagemaker_session=sess,
    entry_point="inference.py",
    source_dir="code",
    py_version="py3",
    env={"MMS_DEFAULT_RESPONSE_TIMEOUT": "500"},
)

In [54]:
compiled_model = sagemaker_model.compile(
    target_instance_family=target_device,
    input_shape=data_shape,
    job_name=compilation_job_name,
    role=role,
    framework=framework.lower(),
    framework_version=framework_version,
    output_path=compiled_model_path,
)

????????????????????????????............................................................................................................................!

In [55]:
predictor = compiled_model.deploy(initial_instance_count=1, instance_type="ml.m5.xlarge")

----!

In [59]:
import numpy as np
import json

im_path = 'dataset/MLPR_augmentedV15/test/ฆ/29_ฆ.jpg'
with open(im_path, "rb") as f:
    payload = f.read()
    payload = bytearray(payload)

print(payload)

response = predictor.predict(payload)
result = json.loads(response.decode())
print("Most likely class: {}".format(np.argmax(result)))

bytearray(b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x08\x06\x06\x07\x06\x05\x08\x07\x07\x07\t\t\x08\n\x0c\x14\r\x0c\x0b\x0b\x0c\x19\x12\x13\x0f\x14\x1d\x1a\x1f\x1e\x1d\x1a\x1c\x1c $.\' ",#\x1c\x1c(7),01444\x1f\'9=82<.342\xff\xdb\x00C\x01\x08\t\t\x0c\x0b\x0c\x18\r\r\x182!\x1c!22222222222222222222222222222222222222222222222222\xff\xc0\x00\x11\x08\x000\x00$\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1f\x00\x00\x01\x05\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\xff\xc4\x00\xb5\x10\x00\x02\x01\x03\x03\x02\x04\x03\x05\x05\x04\x04\x00\x00\x01}\x01\x02\x03\x00\x04\x11\x05\x12!1A\x06\x13Qa\x07"q\x142\x81\x91\xa1\x08#B\xb1\xc1\x15R\xd1\xf0$3br\x82\t\n\x16\x17\x18\x19\x1a%&\'()*456789:CDEFGHIJSTUVWXYZcdefghijstuvwxyz\x83\x84\x85\x86\x87\x88\x89\x8a\x92\x93\x94\x95\x96\x97\x98\x99\x9a\xa2\xa3\xa4\xa5\xa6\xa7\xa8\xa9\xaa\xb2\xb3\xb4\xb5\xb6\xb7\xb8\xb9\xba\xc2\xc3\xc4\xc5\xc6\xc7\xc8\xc9\xca\xd

In [60]:
# Load names for ImageNet classes
object_categories = {}
with open("th_chars_classes_48.txt", "r") as f:
    for line in f:
        key, val = line.strip().split(":")
        object_categories[key] = val.strip(" ").strip(",")
print(
    "The label is",
    object_categories[str(np.argmax(result))],
    "with probability",
    str(np.amax(result))[:5],
)

The label is 'ฆ' with probability 0.999
